In [3]:
# paths and info
import os, sys
homeDir = os.environ['HOMEPATH']
jmodDir = os.environ['JMODELICA_HOME']
moLiDir = os.path.join(homeDir, "Documents", "Dymola")

# give the path to directory where package.mo is stored
moLibs = [os.path.join(jmodDir, "ThirdParty\MSL\Modelica"),
          os.path.join(jmodDir, "ThirdParty\MSL\ModelicaServices"),
          os.path.join(moLiDir, "P509-ML", "Modelon"),
          os.path.join(moLiDir, "P513-RegressionTesting", "RegressionTesting"),
          os.path.join(moLiDir, "P513-RegressionTesting", "RegressionTesting"),
          os.path.join(moLiDir, "P501-AC", "ThermoFluidPro"),
          os.path.join(moLiDir, "P501-AC", "AirConditioning"),
          os.path.join(moLiDir, "P501-AC", "AirConditioningTests"),
         ]

print(sys.version)
print(all(os.path.isfile(os.path.join(moLib, "package.mo")) for moLib in moLibs))
print(os.getcwd())

2.7.13 (v2.7.13:a06454b1afa1, Dec 17 2016, 20:53:40) [MSC v.1500 64 bit (AMD64)]
True
C:\Users\matthis.thorade\Documents\Dymola\modelica-buildings\binder


In [3]:
# compile model to fmu
from pymodelica import compile_fmu
model_name = 'AirConditioningTests.AC_Templates.CondReceiverIHXCycle'
my_fmu = compile_fmu(model_name, moLibs, compiler= 'modelica', compiler_options={"msvs_version":"2017"} , target='me', version='2.0', platform='win64', compile_to='.', compiler_log_level='debug:log.txt', jvm_args='-Xmx8g') 

In [ ]:
# simulate the fmu and store results
from pyfmi import load_fmu
myModel = load_fmu(my_fmu)

opts = myModel.simulate_options()
opts['solver'] = "CVode"
opts['ncp'] = 500
opts['result_handling']="file"
opts["CVode_options"]['discr'] = 'BDF'
opts['CVode_options']['iter'] = 'Newton'
opts['CVode_options']['maxord'] = 5
opts['CVode_options']['atol'] = 1e-6
opts['CVode_options']['rtol'] = 1e-6

res = myModel.simulate(start_time=0, final_time=180, options=opts)

In [ ]:
# plotting of the results
import matplotlib.pyplot as plt

time = 'time'
names = ['summary.SpecificCharge', 'summary.P_condenser']

x = res[time]
for name in names:
    y = res[name]
    plt.plot(x, y)
    plt.xlabel('Time (s)')
    plt.ylabel(name)
    plt.savefig(name.replace('.','_')+'.pdf')
    plt.show()